In [1]:
import requests
from bs4 import BeautifulSoup as bs
import lxml
import csv
import time

In [8]:
vacancies = ['аналитик', 'разработчик', 'дизайнер', 'программист', 'devops']

In [9]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
}

In [10]:
links = []

for vac in vacancies:
  # amount of pages
  url = f'https://spb.hh.ru/search/vacancy?text={vac}&order_by=salary_asc&search_field=name'

  html = requests.get(url, headers = headers).text
  soup = bs(html, 'lxml')

  pages_carousel = soup.find('div', class_='pager').find_all('span', class_='pager-item-not-in-short-range')[-1].get_text()
  pages_carousel = int(pages_carousel)
  print(pages_carousel)

  # links
  for i in range(pages_carousel):
    url = f'{url}&page={i}'


    response = requests.get(url, headers = headers) #get запрос на сайт
    response.status_code

    soup = bs(response.content, 'html.parser')

    vacancy_block = soup.find_all('div', class_='vacancy-serp-item-body__main-info')

    for vacancy in vacancy_block:

      link = vacancy.find('a', class_='serp-item__title')['href'] #ссылка
      links.append(link)

40


In [ ]:
FILE_NAME = 'vacancies.csv'
with open(FILE_NAME, mode="w", encoding='utf-8') as w_file:
  file_writer = csv.writer(w_file, delimiter = ";", lineterminator="\r")
  file_writer.writerow(["vac_title_tag", "vacancy_title", "required_experience", "description", "skills", "address", "mode", "link"])

  for link in links:

    skills = []

    print(link)
    url = link
    response = requests.get(url, headers = headers) #get запрос на вакансию
    response.status_code

    time.sleep(1)

    soup = bs(response.content, 'html.parser')
    #print(soup)
    title_header = soup.find('h1', class_='bloko-header-section-1')
    try:
      title = title_header.text
    except Exception as ex:
      title = ' '

    try:
      exp = soup.find('span', attrs={'data-qa': 'vacancy-experience'}).text
    except Exception as ex:
      exp = ' '

    try:
      desc = soup.find('div', class_='vacancy-section').text.replace('\n', ' ').replace('\xa0', ' ').replace('  ', ' ')
    except Exception as ex:
      desc = ' '

    try:
      req_list = soup.find_all('span', class_='bloko-tag__section bloko-tag__section_text')
      for req in req_list:
        skills.append(req.text.replace('\n', ' ').replace('\xa0', ' ').replace('  ', ' '))
    except Exception as ex:
      skills = []

    try:
      address = soup.find('span', attrs={'data-qa': 'vacancy-view-raw-address'}).text
    except Exception as ex:
      try:
        address = soup.find('p', attrs={'data-qa': 'vacancy-view-location'}).text
      except Exception as ex:
        address = ' '

    try:
      mode = soup.find('p', attrs={'data-qa': 'vacancy-view-employment-mode'}).text
    except Exception as ex:
      mode = ' '


    file_writer.writerow([title_header, title, exp, desc, skills, address, mode, link])

https://adsrv.hh.ru/click?b=790799&c=6&place=35&meta=unBWedSIaFqajNCgE0J8EhshWXK4b_FzfajPL1Ez_ZsM-yhiHgCRq-La6HdjbUZkOwmA7qi_lKb42H7xZM2Wg270tt7_qg7g4_p6wqIgCpr8GpcY10o5dTKu2TWX2Pwz_F3NiWCJq9gb4Mf_Fml2BW4gSbT5ilbij2u6CjhgxERUsbC9TnrkWzzv5zmIvWNsOo8kYmz934_Gq2QCY4t2GtjrFt0udY3hpYvsCB6JzoFhXSA7FuNpmLO1HK0zmsXoAoqQ7znE79ghsBEnqM7hHQIqXvMUlNXEuvfwDurMk7MhBfTNDUYyr-HkOKbGMHxuxQ58637YIgrlV8_E8OU6zcLzoaONnWe-EE6k43swL4P9KH32WPKKKMAw92E5DMlAcMHCwYvyhOvI1CPnERPUHEMtYAGoYv-uCMFxp51HXydk_CFnTQoGRAqM58H3CQXvYPI0uFICChIs9RCB__fxnP2jEb2FfjxUJBisbPu9WrgpZ8C4CY9kcyK9u9pr8TNYnnI492cViySNUhvSwaHl4Q%3D%3D&clickType=link_to_vacancy
https://adsrv.hh.ru/click?b=642290&c=10&place=36&meta=_Z8LptWSMXNPztmrVk_9LiiSNtYNzI4dnaXJTFcbNHnO4tmmyOBkVFgOC1FHLKjuTwPasZ6lVqkU4RQ0z51sv_-oBfcywdG6njR2uDFE_jZjqa26hvrthU9YrJssoKI9_2bDEbWelFqsHDn4Ha89Ve99FqDJnNoKe9qximHb5_Od_yCuAiA3Hw5UE0RsNmo_gIzEqkIRELjTtNxTIKCJkK2eJMPDIu4CrhQzBX3CsIiDY2SGN3aMHeFYhz9JTMI9Dl_6WPeswldCmKAGgxMHgPWL0PnWCqf3lv9IsEkGKXow5wWBejQdvWje7L8qRyNSwGEYgt